# Exploratory Analysis of Spatial Data

3. create interactive visualization in bokeh
5. create interactive mask connection

### To-Do:

* fix data source input
* generate Choropleth of attribute
* make combined view
* include masking functionality in static 3 plot visualization

## Imports

In [1]:
%matplotlib inline 

import matplotlib.pyplot as plt
import pysal as ps
import libpysal.api as lp
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
import splot.plot
import bokeh

In [2]:
from pysal.contrib.pdio import read_files

link_to_data = os.path.join('..', 'example_data', 'guerry', 'Guerry.shp')
df = gpd.read_file(link_to_data)

In [3]:
y = df['Donatns'].values
w = lp.Queen.from_dataframe(df)
w.transform = 'r'

In [14]:
import esda

w = lp.Queen.from_dataframe(df)
moran = esda.moran.Moran(y, w)
moran_loc = esda.moran.Moran_Local(y, w)
moran.I

0.3533613255848606

## Exploring Local Autocorrelation interactively with splot

In [5]:
from collections import OrderedDict
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

First, we have to convert geometries from geopandas to bokeh format:

In [15]:
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i, shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j, member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [16]:
lons, lats = gpd_bokeh(df)

Example for adding data to dataframe

In [8]:
df['lons'] = np.array(lons)
df['lats'] = np.array(lats)

In [9]:
lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
df['lag'] = np.array(lag)

In [10]:
df.head()

,CODE_DE,COUNT,AVE_ID_,dept,Region,Dprtmnt,Crm_prs,Crm_prp,Litercy,Donatns,...,Desertn,Instrct,Prsttts,Distanc,Area,Pop1831,geometry,lons,lats,lag
0,01,1.0,49.0,1,E,Ain,28870,15890,37,5098,...,55,46,13,218.372,5762,346.03,"POLYGON ((801150 2092615, 800669 2093190, 8006...","[801150.0, 800669.0, 800688.0, 800780.0, 80058...","[2092615.0, 2093190.0, 2095430.0, 2095795.0, 2...",-0.729709
1,02,1.0,812.0,2,N,Aisne,26226,5521,51,8901,...,82,24,327,65.945,7369,513.00,"POLYGON ((729326 2521619, 729320 2521230, 7292...","[729326.0, 729320.0, 729280.0, 728751.0, 72866...","[2521619.0, 2521230.0, 2518544.0, 2517520.0, 2...",-0.279823
2,03,1.0,1418.0,3,C,Allier,26747,7925,13,10973,...,16,85,34,161.927,7340,298.26,"POLYGON ((710830 2137350, 711746 2136617, 7124...","[710830.0, 711746.0, 712430.0, 712070.0, 71218...","[2137350.0, 2136617.0, 2135212.0, 2134132.0, 2...",0.130625
3,04,1.0,1603.0,4,E,Basses-Alpes,12935,7289,46,2733,...,32,29,2,351.399,6925,155.90,"POLYGON ((882701 1920024, 882408 1920733, 8817...","[882701.0, 882408.0, 881778.0, 881526.0, 87955...","[1920024.0, 1920733.0, 1921200.0, 1922332.0, 1...",-0.693305
4,05,1.0,1802.0,5,E,Hautes-Alpes,17488,8174,69,6962,...,35,7,1,320.280,5549,129.10,"POLYGON ((886504 1922890, 885733 1922978, 8854...","[886504.0, 885733.0, 885479.0, 883061.0, 88262...","[1922890.0, 1922978.0, 1923276.0, 1925266.0, 1...",-0.726089


In [21]:
def plot_Lisa_Cluster_int(moran_loc, p=0.05):    
    sig = 1 * (moran_loc.p_sim < p)
    HH = 1 * (sig * moran_loc.q==1)
    LL = 3 * (sig * moran_loc.q==3)
    LH = 2 * (sig * moran_loc.q==2)
    HL = 4 * (sig * moran_loc.q==4)

    classes = HH + LL + LH + HL
    class_labels = [ 'ns', 'HH', 'LH', 'LL', 'HL']
    labels = [class_labels[i] for i in classes]

    colors5 = [ 'lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors = [colors5[i] for i in classes]

    #TODO: use df as source instead of lons, lats for whole visualization
    #dfsource = ColumnDataSource(data=df)
    
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    
    fig = figure(title="France", toolbar_location='right',
          plot_width=600, plot_height=500, tools=TOOLS)
    fig.patches(lons, lats, fill_alpha=0.8, fill_color=colors,
         line_color="white", line_width=0.7, line_alpha=0.3)#, legend=True)

    # TODO: this is slow and not quite right (even if it works visually)
    for (colr, leg, x, y ) in zip(colors, labels, lons, lats):
        my_plot = fig.patches(x, y, color= colr, legend= leg)

    return fig

fig = plot_Lisa_Cluster_int(moran_loc, p=0.05)
show(fig)

In [18]:
from bokeh.models import Span

y = df['Donatns'].values
w = lp.Queen.from_dataframe(df)
w.transform = 'r'
    
def mplot_interactive(moran_loc, p=0.05):    
    lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
    fit = ps.spreg.OLS(moran_loc.z[:, None], lag[:,None])
    
    if p is not None:
        if not isinstance(moran_loc, esda.moran.Moran_Local):
            raise ValueError("`moran_loc` is not a Moran_Local instance")

        sig = 1 * (moran_loc.p_sim < p)
        HH = 1 * (sig * moran_loc.q==1)
        LL = 3 * (sig * moran_loc.q==3)
        LH = 2 * (sig * moran_loc.q==2)
        HL = 4 * (sig * moran_loc.q==4)
        classes = HH + LL + LH + HL
        
        class_labels = [ '0 ns', '1 HH', '2 LH', '3 LL', '4 HL']
        labels = [class_labels[i] for i in classes]

    colors5 = [ 'lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors = [colors5[i] for i in classes]
    
    #Tools
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    # Vertical line
    vline = Span(location=0, dimension='height', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    # Horizontal line
    hline = Span(location=0, dimension='width', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    
    fig = figure(title="Moran Scatterplot", x_axis_label='attribute', y_axis_label='Spatial Lag', toolbar_location='left', tools=TOOLS)
    fig.scatter(moran_loc.z, lag, color=colors, size=8, fill_alpha=.6)
    fig.renderers.extend([vline, hline])
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    
    return fig
    

fig = mplot_interactive(moran_loc, p=0.05)
show(fig)


In [ ]:
'''
example coloration for Quantiles

bins_q5 = ps.Quantiles(HR90, k=5)

bwr = plt.cm.get_cmap('Reds')
bwr(.76)
c5 = [bwr(c) for c in [0.2, 0.4, 0.6, 0.7, 1.0]]
classes = bins_q5.yb
colors = [c5[i] for i in classes]'''

# Combined visualizations

Static visualization of Moran Scatterplot, LISA cluster map and choropleth map